## Pregunta de Negocio - Analistes de Finances i Risc Crediticio

> ¿Qué **umbrales de saldo** podrían indicar mayor **riesgo de morosidad**?


## Librerías y Carga de datos

In [ ]:
"""Librerías que pueden ser necesarias"""

# Manejo de datos
import pandas as pd                 # Análisis y manipulación de datos en tablas
import numpy as np                  # Cálculos numéricos y matrices
import os                           # Gestión de rutas de archivos

# Visualización de datos
import matplotlib.pyplot as plt     # Gráficos básicos en 2D
import seaborn as sns               # Gráficos estadísticos mejorados
import plotly.express as px         # Gráficos interactivos simplificados
import plotly.graph_objects as go   # Control avanzado de gráficos en Plotly
import plotly.io as pio             # Interfaz de entrada/salida de Plotly
import squarify                     # Visualización de diagramas de "treemap"

# Procesado y análisis
from scipy import stats
from scipy.stats import chi2_contingency, norm, ttest_ind, f_oneway # Pruebas de hipótesis y análisis estadístico
from matplotlib.patches import Patch
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report,roc_auc_score, confusion_matrix,roc_curve, auc

import warnings
warnings.filterwarnings("ignore")#ignorar advertencias


#Paleta de colores oficial
custom_palette = [ "#2CCED1", "#5F6F81", "#AABBC8", "#DCE3EA", "#2E2F36"]

In [ ]:
df = pd.read_csv("../Data/df_250526.csv")

### Tratamiento previo de los datos
#### Para saldo que tiene muchos outliers, transformación logarítmica 
Reduce la asimetria de la variable y mejora la normalidad de la distribución, conservando el orden relativo de los valores.
#### Para default que tiene mucho sesgo, balanceo de clases
esto lo hacemos dentro de la función de entrenamiento del modelo, para que no afecte a la métrica de evaluación. (class_weight='balanced')

In [ ]:

# Evita problemas con valores negativos o cero
'''df['balance_log'] = np.log(df['balance'] + 1)
descartado me da valores NaN para los negativos'''

# Solo mantener registros con balance positivo
'''df['balance_log'] = df['balance'].apply(lambda x: np.log(x + 1) if x >= 0 else np.nan) 
Tambien descartado pues me quita los negativos y sigo con valores NaN'''
# transformacion simetrica logaritmica , para evitar valores NaN y mejorar la normalidad  conservar el signo de los valores negativos.
'''df['balance_log'] = df['balance'].apply(lambda x: np.sign(x) * np.log1p(abs(x))) 
Otro descarte pues me acumula artificialmente entorno a 0 lo que me da un pico en 0 y no me sirve para nada'''

#logaritmo hiperbólico inverso, para ver si logro eliminar el pico en 0
df['balance_log'] = np.arcsinh(df['balance'])
'''Al final acepte que hubiera pico en 0 pues en realidad los datos se concentran 
entorno a este valor por lo que me parece razonable
y no me afecta a la normalidad de la variable.'''

## KPIs

1. SALDO MEDIO GENERAL
2. PORCENTAJE DE CLIENTES POR TIPO DE PAQUETE CONTRATADO
3. SALDO MEDIO POR CATEGORÍA DE CRÉDITO
4. PROBABILIDAD DE INCUMPLIMIENTO (PD) GLOBAL
5. ANÁLISIS POR DECILES DE SALDO

In [ ]:
#  1. SALDO MEDIO GENERAL
saldo_medio_global = df['balance'].mean()
print(f"Saldo medio general: {round(saldo_medio_global / 1000, 1)} k€")

In [ ]:
#  2. PORCENTAJE DE CLIENTES POR TIPO DE PAQUETE CONTRATADO
total_clientes = len(df)

pct_hipoteca = 100 * df['housing'].value_counts().get(True, 0) / total_clientes
pct_prestamo = 100 * df['loan'].value_counts().get(True, 0) / total_clientes
pct_ambos = 100 * len(df[df['loan'] & df['housing']]) / total_clientes
pct_deposito = 100 * df['deposit'].value_counts().get(True, 0) / total_clientes

print(f"% con hipoteca: {pct_hipoteca:.2f}%")
print(f"% con préstamo: {pct_prestamo:.2f}%")
print(f"% con ambos: {pct_ambos:.2f}%")
print(f"% contratación de depósito: {pct_deposito:.2f}%")

In [ ]:
#  3. SALDO MEDIO POR CATEGORÍA DE CRÉDITO
def categorize_credit(row):
    if not row['loan'] and not row['housing']:
        return 'Sin créditos'
    elif not row['loan'] and row['housing']:
        return 'Sólo hipoteca'
    elif row['loan'] and not row['housing']:
        return 'Sólo préstamo'
    else:
        return 'Ambos'

df['categoria_credito'] = df.apply(categorize_credit, axis=1)

balance_promedio = df.groupby('categoria_credito')['balance'].mean()
clientes_por_categoria = df['categoria_credito'].value_counts()
df_balance = pd.DataFrame({
    'categoria_credito': balance_promedio.index,
    'balance_promedio': balance_promedio.values,
    'clientes_por_categoria': clientes_por_categoria[balance_promedio.index].values
})
df_balance['pct_clientes'] = (df_balance['clientes_por_categoria'] / total_clientes * 100).round(1)

orden = ['Sin créditos', 'Sólo hipoteca', 'Sólo préstamo', 'Ambos']
df_balance['categoria_credito'] = pd.Categorical(df_balance['categoria_credito'], categories=orden, ordered=True)
df_balance = df_balance.sort_values('categoria_credito').reset_index(drop=True)

print(df_balance)

# Pie chart de distribución por tipo de crédito
plt.figure(figsize=(5, 5))
plt.pie(
    df_balance['pct_clientes'],
    labels=df_balance['categoria_credito'],
    labeldistance=1.05,
    autopct='%1.1f%%',
    colors=custom_palette,
    wedgeprops={'linewidth': 1, 'edgecolor': 'white'}
)
plt.title("Distribución de clientes por tipo de crédito contratado")
plt.show()

In [ ]:
#  4. PROBABILIDAD DE INCUMPLIMIENTO (PD) GLOBAL
pd_global = df['default'].mean()
print("PD global =", (pd_global*100).round(2), "%")

In [ ]:
#  5. ANÁLISIS POR DECILES DE SALDO
df['balance_decile'] = pd.qcut(df['balance'], q=10, labels=False) + 1

default_rates = (
    df.groupby('balance_decile')['default']
    .agg(['mean', 'count'])
    .rename(columns={'mean': 'default', 'count': 'clientes'})
    .reset_index()
)
default_rates['default'] = default_rates['default'].round(4)
default_rates['indice_riesgo'] = (default_rates['default'] / pd_global).round(2)
default_rates['porcentaje_clientes'] = (default_rates['clientes'] / total_clientes * 100).round(2)

def categorize_risk(rate):
    if rate > 0.10:
        return 'Muy alto'
    elif rate > 0.05:
        return 'Alto'
    elif rate > 0.02:
        return 'Moderado'
    return 'Bajo'

default_rates['risk_level'] = default_rates['default'].apply(categorize_risk)

# Porcentaje de clientes por categoría de riesgo
porcentaje_riesgo = (
    default_rates.groupby('risk_level')['porcentaje_clientes']
    .sum()
    .reindex(['Muy alto', 'Alto', 'Moderado', 'Bajo'])
    .fillna(0)
    .reset_index()
)
porcentaje_riesgo['porcentaje_clientes'] = porcentaje_riesgo['porcentaje_clientes'].astype(int)

print(porcentaje_riesgo)

## Analisis descriptivo de saldo segun default
comparar la distribucion de saldo entre los clientes que han tenido default y los que no.

```python

In [ ]:
promedio = df.groupby('default')['balance'].mean().round(2)
mediana = df.groupby('default')['balance'].median().round(2)
minimo = df.groupby('default')['balance'].min().round(2)
maximo = df.groupby('default')['balance'].max()
Descriptiva = pd.DataFrame({
    'Promedio': promedio,
    'Mediana': mediana,
    'Mínimo': minimo,
    'Máximo': maximo
})

promedio = df.groupby('default')['balance_log'].mean().round(2)
mediana = df.groupby('default')['balance_log'].median().round(2)
minimo = df.groupby('default')['balance_log'].min().round(2)
maximo = df.groupby('default')['balance_log'].max()
Descriptiva_log = pd.DataFrame({
    'Promedio': promedio,
    'Mediana': mediana,
    'Mínimo': minimo,
    'Máximo': maximo
})
print("Análisis descriptivo de la variable balance")
print(Descriptiva)
print("Analisis descriptivo de la variable balance_log")
print(Descriptiva_log)


#### Comparativa de distribuciones de saldo original y con logaritmo

In [ ]:
#  Crear rangos de balance para agrupar clientes en grupos iguales
df['balance_decile'] = pd.qcut(df['balance'], q=15, labels=False)
df['balance_log_decile'] = pd.qcut(df['balance_log'], q=15, labels=False)
# Visualizar la distribución de la variable 'balance' y su transformación logarítmica
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.histplot(df['balance'], bins=50, kde=True, ax=axes[0], color='red')
axes[0].set_title("Distribución original de saldo")

sns.histplot(df['balance_log'], bins=50, kde=True, ax=axes[1], color='green')
axes[1].set_title("Distribución logarítmica de saldo")

plt.tight_layout()
plt.show()


### Agupacion de saldos en rangos

In [ ]:


# Calcular la tasa de incumplimiento (default) por cada grupo de saldo
default_rates = (df.groupby('balance_decile')['default'].mean().reset_index()) #la media aritmética en ese grupo equivale a la proporción (o tasa) de incumplimiento
default_rates['default'] = default_rates['default'].round(4) 
default_rates['balance_decile'] = default_rates['balance_decile'] + 1
Tasa_global_impagos = df['default'].mean()
print(f"Tasa global de incumplimiento: {Tasa_global_impagos:.4f}")
# Calculamos el indice de riesgo por decil riesgo/riesgo global
default_rates['indice_riesgo'] = default_rates['default'] / Tasa_global_impagos
default_rates['indice_riesgo'] = default_rates['indice_riesgo'].round(2)

# === Balance transformado (logarítmico o arcsinh) ===
default_rates_log = df.groupby('balance_log_decile')['default'].mean().reset_index()
default_rates_log['default'] = default_rates_log['default'].round(4)
default_rates_log['balance_log_decile'] += 1
default_rates_log['indice_riesgo'] = (default_rates_log['default'] / Tasa_global_impagos).round(2)
df['balance_log_decile_range'] = pd.qcut(df['balance_log'], q=15, duplicates='drop')

#  Mostrar resultados en consola
print("Tasa de incumplimiento por decil de saldo:")
print(default_rates)
print('Tasa de incumplimiento por decil de saldo logarítmico:')
print(default_rates_log)
df['balance_decile_range'] = pd.qcut(df['balance'], q=15)

# Obtener los intervalos únicos en orden
balance_ranges = df['balance_decile_range'].cat.categories
decile_labels = [
    f"{i+1}\n(€{int(interval.left)} - €{int(interval.right)}]"
    for i, interval in enumerate(balance_ranges)
]

# Etiquetas para los deciles logarítmicos
balance_log_ranges = df['balance_log_decile_range'].cat.categories
decile_log_labels = [
    f"{i+1}\n({round(interval.left, 2)} - {round(interval.right, 2)}]"
    for i, interval in enumerate(balance_log_ranges)
]

#default_rates['risk_level'] = default_rates['default'].apply(categorize_risk)
plt.figure(figsize=(12, 6))
sns.barplot(data=default_rates, x='balance_decile', y='default', palette=custom_palette, edgecolor='black')
plt.axhline(Tasa_global_impagos, color='#4A3369', linestyle='--', label='Tasa global')
plt.xticks(ticks=range(15), labels=decile_labels, rotation=45, ha='right')
plt.title('Tasa de incumplimiento por rango de saldo original')
plt.xlabel('Rango de saldo')
plt.ylabel('Tasa de incumplimiento')
plt.legend(title='Riesgo')
plt.tight_layout()
plt.show()


# Gráfico para balance_log
plt.figure(figsize=(12, 6))
sns.barplot(data=default_rates_log, x='balance_log_decile', y='default', palette=custom_palette, edgecolor='black')
plt.axhline(Tasa_global_impagos, color='#4A3369', linestyle='--', label='Tasa global')
plt.xticks(ticks=range(len(decile_log_labels)), labels=decile_log_labels, rotation=45, ha='right')
plt.title('Tasa de incumplimiento por rango de saldo transformado')
plt.xlabel('Rango de saldo (balance_log)')
plt.ylabel('Tasa de incumplimiento')
plt.legend(title='Riesgo')
plt.tight_layout()
plt.show()



Según los datos más recientes del Banco de España, la tasa de morosidad de la banca española cerró 2024 en un 3,32%, la más baja desde finales de 2008 . Este indicador refleja el porcentaje de préstamos impagados en relación con el total de créditos concedidos.

### Preparacion entrenamiento

In [ ]:
# data frame boleano
df_bool = df[['default', 'housing', 'loan', 'deposit', 'balance']].copy()


 

for col in ['default','housing', 'loan', 'deposit']:
    df_bool[col] = df_bool[col].astype(int)

# Definir las variables predictoras y la variable objetivo
X = df_bool[[ 'housing', 'loan']]
y = df_bool['default']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



### Regresion logistica

In [ ]:
log_model = LogisticRegression(class_weight='balanced')
log_model.fit(X_train, y_train)
y_pred = log_model.predict(X_test)
y_prob = log_model.predict_proba(X_test)[:,1]
# Resultados
print("=== Regresión Logística ===")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f"AUC: {roc_auc_score(y_test, y_prob):.4f}")

coefs = pd.Series(log_model.coef_[0], index=X.columns)
print("Coeficientes del modelo:")
print(coefs.sort_values(ascending=False))



### Arbol de decision

In [ ]:
tree_model = DecisionTreeClassifier(max_depth=4,class_weight='balanced', random_state=42)
tree_model.fit(X_train, y_train)

y_pred_tree = tree_model.predict(X_test)
y_prob_tree = tree_model.predict_proba(X_test)[:,1]

print("=== Árbol de Decisión ===")
print(confusion_matrix(y_test, y_pred_tree))
print(classification_report(y_test, y_pred_tree))
print(f"AUC: {roc_auc_score(y_test, y_prob_tree):.4f}")

# Visualización del árbol
plt.figure(figsize=(20, 10))
plot_tree(tree_model, feature_names=X.columns, class_names=["No Default", "Default"], filled=True)
plt.title("Árbol de Decisión")
plt.show()

### ROC Curve

In [ ]:
df_ROC = pd.DataFrame({ 
    'y_true': y_test,
    'p_logreg': y_prob,
    'p_tree': y_prob_tree
})

print(df_ROC.head())
# Calcula curvas ROC y AUC
fpr_lr, tpr_lr, _ = roc_curve(df_ROC['y_true'], df_ROC['p_logreg'])
fpr_dt, tpr_dt, _ = roc_curve(df_ROC['y_true'], df_ROC['p_tree'])
auc_lr = auc(fpr_lr, tpr_lr)
auc_dt = auc(fpr_dt, tpr_dt)

# Dibuja
plt.figure()
plt.plot(fpr_lr, tpr_lr, label=f'LogReg (AUC = {auc_lr:.3f})')
plt.plot(fpr_dt, tpr_dt, label=f'DecTree (AUC = {auc_dt:.3f})')
plt.plot([0,1], [0,1], linestyle='--', label='Random baseline')
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('ROC Curve Comparativa')
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()


### Lift Chart (ganancias)

In [ ]:
def cumulative_gains(y_true, y_score, n_bins=10):
    dfc = pd.DataFrame({'y': y_true, 'score': y_score})
    dfc = dfc.sort_values('score', ascending=False).reset_index(drop=True)
    dfc['cum_default'] = dfc['y'].cumsum()
    dfc['cum_pct_obs'] = (np.arange(len(dfc)) + 1) / len(dfc)
    dfc['cum_pct_def'] = dfc['cum_default'] / dfc['y'].sum()
    return dfc

# Genera curvas
cg_lr = cumulative_gains(df_ROC['y_true'], df_ROC['p_logreg'])
cg_dt = cumulative_gains(df_ROC['y_true'], df_ROC['p_tree'])

plt.figure()
plt.plot(cg_lr['cum_pct_obs'], cg_lr['cum_pct_def'], label='LogReg')
plt.plot(cg_dt['cum_pct_obs'], cg_dt['cum_pct_def'], label='DecTree')
plt.plot([0,1], [0,1], linestyle='--', label='Random baseline')
plt.xlabel('% Observaciones (desde mayor score)')
plt.ylabel('% Default acumulado')
plt.title('Lift Chart / Cumulative Gains')
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

### Curva riesgo saldo

In [ ]:
iso = IsotonicRegression(out_of_bounds='clip')
iso.fit(df_ROC['p_logreg'], df_ROC['y_true'])

# Probabilidades recalibradas
df_ROC['p_logreg_calibrated'] = iso.transform(df_ROC['p_logreg'])

# Definir bins de balance (p. ej. 15 rangos )
df_ROC['balance_bin'] = pd.cut(df['balance'], bins=10)

# Para cada bin, promedio de probabilidad y tasa real
summary = df_ROC.groupby('balance_bin').agg(
    avg_pred_proba = ('p_logreg_calibrated', 'mean'),
    real_default_rate = ('y_true', 'mean')
).reset_index()

plt.figure()
plt.plot(summary['balance_bin'].astype(str), summary['avg_pred_proba'], marker='o', label='Probabilidad Predicha')
plt.plot(summary['balance_bin'].astype(str), summary['real_default_rate'], marker='x', label='Tasa Real Default')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Rango de Saldo')
plt.ylabel('Proporción')
plt.title('Curva de Riesgo vs. Saldo')
plt.legend()
plt.tight_layout()
plt.show()

## Regresión logística para predicción del default en función del balance
Estimar directamente cómo balance afecta la probabilidad de incumplimiento (default)

In [ ]:
# convertir variables binarias
df[['default']].astype(int)

In [ ]:
X = df[['balance']]     # predictor
y = df['default']       # objetivo --> variable binaria

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
model = LogisticRegression()
model.fit(X_train, y_train)

# Coeficientes y el intercepto 
print(model.coef_)
print("Intercepto:", model.intercept_[0])
print("Coeficiente balance:", model.coef_[0][0])

In [ ]:
# Predecir probabilidad en función del saldo --> crear un rango de valores uniforme para ver la variación del default
balances = np.linspace(df['balance'].min(), df['balance'].max(), 1000).reshape(-1, 1)
proba_default = model.predict_proba(balances)[:, 1]

In [ ]:
# Visualizar la curva
plt.plot(balances, proba_default)
plt.xlabel('Balance (€)')
plt.ylabel('Probabilidad de Default')
plt.title('Probabilidad de default según el saldo')
plt.grid()
plt.show()

##### Análisis del resultadi
- La curva muestra una alta probabilidad de default para saldos cercanos o inferiores a 0.
- Cuando el saldo aumenta, la probabilidad baja a 0.
- Confirma que el saldo es un fuerte predictor del riesgo de morosidad

In [ ]:
# Saldo a partir del cual la probabilidad de default cae por debajo del 5%:
probab_limite = 0.05
umbral_saldo = balances[proba_default >= probab_limite].max()
print(f"Umbral de riesgo alto: balance < {umbral_saldo:.2f} €")

## Regresión logística para predicción del default en función de balance, loan y housing
Estimar directamente cómo balance, loan y housing afectan en conjunto a la probabilidad de incumplimiento (default)

In [ ]:
# convertir variables binarias
df[['default', 'housing', 'loan']].astype(int)

In [ ]:
X = df[['balance', 'housing', 'loan']]     # predictores
y = df['default']       # objetivo --> variable binaria

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
model = LogisticRegression()
model.fit(X_train, y_train)

# Coeficientes y el intercepto 
print(model.coef_)
print("Intercepto:", model.intercept_[0])
print("Coeficiente balance:", model.coef_[0][0])
print("Coeficiente housing:", model.coef_[0][1])
print("Coeficiente loan:", model.coef_[0][2])

#### Sin créditos

In [ ]:
# Predecir probabilidad en función del saldo --> crear un rango de valores uniforme para ver la variación del default
balances_sin = np.linspace(df['balance'].min(), df['balance'].max(), 1000).reshape(-1, 1)
housing_val_sin = np.zeros_like(balances_sin)  # todos 0
loan_val_sin = np.zeros_like(balances_sin)  # todos 0

X_plot_sin = np.hstack([balances_sin, housing_val_sin, loan_val_sin])
proba_default_sin = model.predict_proba(X_plot_sin)[:, 1]

# Visualizar la curva
plt.plot(balances_sin, proba_default_sin)
plt.xlabel('Balance (€)')
plt.ylabel('Probabilidad de Default')
plt.title('Probabilidad de default (housing=0, loan=0)')
plt.grid()
plt.show()

In [ ]:
# Saldo a partir del cual la probabilidad de default cae por debajo del 5%:
probab_limite = 0.05
umbral_saldo_sin = balances_sin[proba_default_sin >= probab_limite].max()
print(f"Sin créditos. Umbral de riesgo alto: balance < {umbral_saldo_sin:.2f} €")

#### Solo hipoteca

In [ ]:
# Predecir probabilidad en función del saldo --> crear un rango de valores uniforme para ver la variación del default
balances_hipoteca = np.linspace(df['balance'].min(), df['balance'].max(), 1000).reshape(-1, 1)
housing_val_hipoteca = np.ones_like(balances_hipoteca)  # todos 1
loan_val_hipoteca = np.zeros_like(balances_hipoteca)  # todos 0

X_plot_hipoteca = np.hstack([balances_hipoteca, housing_val_hipoteca, loan_val_hipoteca])
proba_default_hipoteca = model.predict_proba(X_plot_hipoteca)[:, 1]

# Visualizar la curva
plt.plot(balances_hipoteca, proba_default_hipoteca)
plt.xlabel('Balance (€)')
plt.ylabel('Probabilidad de Default')
plt.title('Probabilidad de default (housing=1, loan=0)')
plt.grid()
plt.show()

In [ ]:
# Saldo a partir del cual la probabilidad de default cae por debajo del 5%:
probab_limite = 0.05
umbral_saldo_hipoteca = balances_hipoteca[proba_default_hipoteca >= probab_limite].max()
print(f"Solo hipoteca. Umbral de riesgo alto: balance < {umbral_saldo_hipoteca:.2f} €")

#### Solo préstamo

In [ ]:
# Predecir probabilidad en función del saldo --> crear un rango de valores uniforme para ver la variación del default
balances_prestamo = np.linspace(df['balance'].min(), df['balance'].max(), 1000).reshape(-1, 1)
housing_val_prestamo = np.zeros_like(balances_prestamo)  # todos 0
loan_val_prestamo = np.ones_like(balances_prestamo)  # todos 1

X_plot_prestamo = np.hstack([balances_prestamo, housing_val_prestamo, loan_val_prestamo])
proba_default_prestamo = model.predict_proba(X_plot_prestamo)[:, 1]

# Visualizar la curva
plt.plot(balances_prestamo, proba_default_prestamo)
plt.xlabel('Balance (€)')
plt.ylabel('Probabilidad de Default')
plt.title('Probabilidad de default (housing=0, loan=1)')
plt.grid()
plt.show()

In [ ]:
# Saldo a partir del cual la probabilidad de default cae por debajo del 5%:
probab_limite = 0.015
umbral_saldo_prestamo = balances_prestamo[proba_default_prestamo >= probab_limite].max()
print(f"Solo préstamo. Umbral de riesgo alto: balance < {umbral_saldo_prestamo:.2f} €")

#### Ambos

In [ ]:
# Predecir probabilidad en función del saldo --> crear un rango de valores uniforme para ver la variación del default
balances_ambos = np.linspace(df['balance'].min(), df['balance'].max(), 1000).reshape(-1, 1)
housing_val_ambos = np.ones_like(balances_ambos)  # todos 
loan_val_ambos = np.ones_like(balances_ambos)  # todos 1

X_plot_ambos = np.hstack([balances_ambos, housing_val_ambos, loan_val_ambos])
proba_default_ambos = model.predict_proba(X_plot_ambos)[:, 1]

# Visualizar la curva
plt.plot(balances_ambos, proba_default_ambos)
plt.xlabel('Balance (€)')
plt.ylabel('Probabilidad de Default')
plt.title('Probabilidad de default (housing=1, loan=1)')
plt.grid()
plt.show()

In [ ]:
# Saldo a partir del cual la probabilidad de default cae por debajo del 5%:
probab_limite = 0.05
umbral_saldo_ambos = balances_ambos[proba_default_ambos >= probab_limite].max()
print(f"Con ambos. Umbral de riesgo alto: balance < {umbral_saldo_ambos:.2f} €")

In [ ]:
print(f"Sin créditos: {umbral_saldo_sin:.2f} €")
print(f"Solo hipoteca: {umbral_saldo_hipoteca:.2f} €")
print(f"Solo préstamo: {umbral_saldo_prestamo:.2f} €")
print(f"Con ambos: {umbral_saldo_ambos:.2f} €")

#### Intento Graficas Conjuntas

In [ ]:
conditions = [
    (~df['housing']) & (~df['loan']),
    (df['loan']) & (~df['housing']),
    (df['housing']) & (~df['loan']),
    (df['housing']) & (df['loan'])
]
choices = ['sin deudas', 'con préstamo', 'con hipoteca', 'con ambos']
df['debt_status'] = np.select(conditions, choices)

# 2. Deciles de saldo y rangos
df['decile'] = pd.qcut(df['balance'], 10, labels=False) + 1
_, bins = pd.qcut(df['balance'], 10, retbins=True)
balance_ranges = pd.IntervalIndex.from_breaks(bins)

# 3. Función y mapa de colores
def categorize_risk(rate):
    if rate > 0.10:
        return 'Muy alto'
    elif rate > 0.05:
        return 'Alto'
    elif rate > 0.02:
        return 'Moderado'
    else:
        return 'Bajo'

# Orden explícito para la leyenda
levels = ['Muy alto', 'Alto', 'Moderado', 'Bajo']
color_map = {
    'Muy alto': '#4A3369',
    'Alto':    '#4FD0DB',
    'Moderado':'#8FA0A3',
    'Bajo':    '#30353F'
}

# 4. Gráfica 2: 2×2 subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharey=True)
axes = axes.flatten()
groups = ['sin deudas', 'con préstamo', 'con hipoteca', 'con ambos']

for ax, grp in zip(axes, groups):
    grp_df = df[df['debt_status'] == grp]
    rates = grp_df.groupby('decile')['default'].mean().reset_index()
    rates['risk_level'] = rates['default'].apply(categorize_risk)
    
    # Dibujar barras, asignando label solo la primera vez por nivel
    seen = set()
    for idx, row in rates.iterrows():
        lvl = row['risk_level']
        lbl = lvl if lvl not in seen else None
        ax.bar(idx, row['default'], color=color_map[lvl], label=lbl)
        seen.add(lvl)
    
    # Línea de tasa global
    ax.axhline(df['default'].mean(), linestyle='--', color='black', label='Tasa global')
    
    # Etiquetas
    ax.set_title(grp)
    ax.set_xticks(range(10))
    ax.set_xticklabels(
        [f"{i+1}\n(€{int(interval.left)} - €{int(interval.right)}]"
         for i, interval in enumerate(balance_ranges)],
        rotation=45, ha='right'
    )
    ax.set_ylim(0, rates['default'].max() * 1.1)

# Leyenda global
fig.legend(title='Nivel de riesgo', loc='upper right')
fig.suptitle('Tasa de incumplimiento por decil de saldo (por grupo)', y=1.02)
fig.text(0.5, 0.04, 'Decil de saldo', ha='center')
fig.text(0.04, 0.5, 'Tasa de incumplimiento', va='center', rotation='vertical')

plt.tight_layout()
plt.show()

In [ ]:
# 2. Definir bins para balance
n_bins = 15
bins = np.linspace(df['balance'].min(), df['balance'].max(), n_bins + 1)

# 3. Calcular tasa de default por bin y por grupo
rate_by_group = {}
for group in choices:
    grp = df[df['debt_status'] == group].copy()
    grp['bin'] = pd.cut(grp['balance'], bins=bins, include_lowest=True)
    rates = grp.groupby('bin')['default'].mean().reset_index()
    rates['bin_center'] = rates['bin'].apply(lambda x: (x.left + x.right) / 2)
    rate_by_group[group] = rates

# 4. Graficar curvas
plt.figure(figsize=(10, 6))
for group, rates in rate_by_group.items():
    plt.plot(rates['bin_center'], rates['default'], label=group)

# 5. Calcular y marcar umbrales (cuando tasa > 5%)
for group, rates in rate_by_group.items():
    above_5 = rates[rates['default'] > 0.05]
    if not above_5.empty:
        thresh = above_5.iloc[0]
        plt.scatter(thresh['bin_center'], thresh['default'])
        plt.text(thresh['bin_center'], thresh['default'],
                 f"{group}\n€{thresh['bin_center']:.0f}", 
                 va='bottom', ha='right')

# 6. Personalización
plt.xlabel('Saldo (€)')
plt.ylabel('Tasa de incumplimiento')
plt.title('Curvas de tasa de default vs. saldo por grupo')
plt.legend(title='Grupo')
plt.tight_layout()
plt.show()

In [ ]:
n_quantiles = 5
# 4. Graficar curvas y puntos de corte
plt.figure(figsize=(10, 6))
thresholds = {}

for group in choices:
    grp = df[df['debt_status'] == group].copy()
    # Quantile bins por grupo
    grp['bin'], bins = pd.qcut(grp['balance'], q=n_quantiles, duplicates='drop', retbins=True)
    # Calcular tasa de default y centroide del bin
    rates = grp.groupby('bin')['default'].mean().reset_index()
    rates['bin_center'] = grp.groupby('bin')['balance'].median().values

    # Dibujar curva
    plt.plot(rates['bin_center'], rates['default'], marker='o', label=group)
    
    # Punto de umbral >1.5%
    above5 = rates[rates['default'] > 0.015]
    if not above5.empty:
        pts = above5.iloc[0]
        thresholds[group] = pts['bin_center']
        plt.scatter(pts['bin_center'], pts['default'], s=100)
        plt.text(pts['bin_center'], pts['default'],
                 f"€{pts['bin_center']:.0f}", va='bottom', ha='right')

# 5. Personalización final
plt.xlabel('Saldo (€)')
plt.ylabel('Tasa de incumplimiento')
plt.title('Curvas de default vs. saldo por grupo ')
plt.legend(title='Grupo')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# 6. Imprimir umbrales detectados
print("Umbrales de saldo donde tasa > 1.5% por grupo:")
for grp, val in thresholds.items():
    print(f" - {grp}: €{val:.0f}")